# Data Preparation

> Before diving into the analysis, it's essential to clean and preprocess the data. In this section, we will:

1. **Load the data** from the source file.

2. **Inspect** the data for missing or incorrect values.

3. **Clean** the data by handling missing or incorrect values, and standardize formats.

4. **Transform** the data, if necessary, to create new features or better representations of existing features for our analysis.

> By carefully preparing the data, we can ensure that our analysis is accurate and reliable.

## Important Fields for Time-based Analysis to Reduce Critical Traffic Collisions

The following fields have been selected for their relevance in conducting a time-based analysis with the goal of reducing critical traffic collisions by 10%. Understanding the trends and patterns in these fields can help inform data-driven policies and initiatives aimed at reducing traffic fatalities and injuries.

| Column Name                   | Description                                                | Type        |
|-------------------------------|------------------------------------------------------------|-------------|
| CRASH DATE                    | Occurrence date of collision                               | Date & Time |
| CRASH TIME                    | Occurrence time of collision                               | Plain Text  |
| BOROUGH                       | Borough where collision occurred                           | Plain Text  |
| LATITUDE                      | Latitude coordinate for Global Coordinate System           | Number      |
| LONGITUDE                     | Longitude coordinate for Global Coordinate System          | Number      |
| NUMBER OF PERSONS INJURED     | Number of persons injured                                  | Number      |
| NUMBER OF PERSONS KILLED      | Number of persons killed                                   | Number      |
| NUMBER OF PEDESTRIANS INJURED | Number of pedestrians injured                              | Number      |
| NUMBER OF PEDESTRIANS KILLED  | Number of pedestrians killed                               | Number      |
| NUMBER OF CYCLIST INJURED     | Number of cyclists injured                                 | Number      |
| NUMBER OF CYCLIST KILLED      | Number of cyclists killed                                  | Number      |
| NUMBER OF MOTORIST INJURED    | Number of vehicle occupants injured                        | Number      |
| NUMBER OF MOTORIST KILLED     | Number of vehicle occupants killed                         | Number      |
| CONTRIBUTING FACTOR VEHICLE 1 | Factors contributing to the collision for designated vehicle| Plain Text  |
| CONTRIBUTING FACTOR VEHICLE 2 | Factors contributing to the collision for designated vehicle| Plain Text  |
| VEHICLE TYPE CODE 1           | Type of vehicle involved                                   | Plain Text  |
| VEHICLE TYPE CODE 2           | Type of vehicle involved                                   | Plain Text  |

> **CRASH DATE and CRASH TIME**: 

Analyzing the date and time of collisions can reveal patterns, such as specific days of the week or times of the day with higher collision rates. This information can help target interventions and allocate resources effectively.
> **BOROUGH, LATITUDE, and LONGITUDE**: 

Geographical data allows for the identification of high-risk areas and hotspots, as well as the evaluation of local policies and infrastructure.
> **NUMBER OF PERSONS INJURED, NUMBER OF PERSONS KILLED, NUMBER OF PEDESTRIANS INJURED, NUMBER OF PEDESTRIANS KILLED, NUMBER OF CYCLIST INJURED, NUMBER OF CYCLIST KILLED, NUMBER OF MOTORIST INJURED, NUMBER OF MOTORIST KILLED**: 

These fields provide crucial information on the severity of collisions and their impact on different road users. This data can be used to prioritize interventions and track progress towards reducing critical traffic collisions.
> **CONTRIBUTING FACTOR VEHICLE 1 and CONTRIBUTING FACTOR VEHICLE 2**: 

Understanding the factors contributing to collisions is essential for identifying targeted interventions and addressing the root causes of traffic collisions.
> **VEHICLE TYPE CODE 1 and VEHICLE TYPE CODE 2**: 

Vehicle types can play a significant role in the severity and frequency of collisions. Analyzing vehicle type data can help identify specific vehicle types that may be overrepresented in critical traffic collisions and inform interventions tailored to those types.


In [ ]:
# ((Codeblock One))

import pandas as pd

In [ ]:
# ((Codeblock Two))

#Load the NYPD Motor Collisions file
df = pd.read_csv("/Users/Administrator/Documents/Motor_Vehicle_Collisions_-_Crashes.csv")

In [ ]:
# ((Codeblock Three))

df.info(verbose = True, show_counts = True)

## Analyst Notes

**Total records:** 1.9 million

**Records with Borough and Zip Code info:** 1.3 million

This means we're missing location records (Borough and Zip Code) for approximately 600k+ entries, or ~30% of the data.

> **Note:** We could potentially use reverse geocoding with the `GeoPY` library to find the address (Borough and Zip Code) for records that have a location (Latitude and Longitude). However, this process is time-consuming and may not be feasible for this exercise. As a result, we won't be conducting reverse geocoding in our analysis.

## Analyst Notes
## Renaming Fields for Consistency
To ensure that our dataset aligns with our internal data model and follows a common structure, we will rename certain fields. This will make it easier to integrate our analysis with other data sources and facilitate collaboration across teams.

>Follow these steps to rename fields:

Review the current field names and compare them with the internal data model's requirements.
Identify any discrepancies or inconsistencies in field names.
Rename the fields to match the internal data model, ensuring that they adhere to a common naming convention.
By standardizing field names, we can improve data quality and make our analysis more efficient and reliable.

> **Note:** Remember to document any changes made to field names, as this will help maintain transparency and facilitate future data management tasks.

In [ ]:
# ((Codeblock Four))

df.rename(columns = {  'CRASH DATE' : 'date',
                       'CRASH TIME' : 'time',
                       'BOROUGH' : 'borough',
                       'ZIP CODE' : 'zip_code',
                       'LATITUDE' : 'latitude',
                       'LONGITUDE' : 'longitude',
                       'LOCATION' : 'location',
                       'ON STREET NAME'    : 'street_on',
                       'CROSS STREET NAME' : 'street_cross',
                       'OFF STREET NAME'   : 'street_off',
                       'NUMBER OF PERSONS INJURED'     : 'total_injured',
                       'NUMBER OF PERSONS KILLED'      : 'total_fatality',
                       'NUMBER OF PEDESTRIANS INJURED' : 'ped_injured',
                       'NUMBER OF PEDESTRIANS KILLED'  : 'ped_fatality',
                       'NUMBER OF CYCLIST INJURED'     : 'cyc_injured',
                       'NUMBER OF CYCLIST KILLED'      : 'cyc_fatality',
                       'NUMBER OF MOTORIST INJURED'    : 'moto_injured',
                       'NUMBER OF MOTORIST KILLED'     : 'moto_fatality',
                       'CONTRIBUTING FACTOR VEHICLE 1' : 'veh_factor_1',
                       'CONTRIBUTING FACTOR VEHICLE 2' : 'veh_factor_2',
                       'CONTRIBUTING FACTOR VEHICLE 3' : 'veh_factor_3',
                       'CONTRIBUTING FACTOR VEHICLE 4' : 'veh_factor_4',
                       'CONTRIBUTING FACTOR VEHICLE 5' : 'veh_factor_5',
                       'UNIQUE KEY' : 'unique_key',
                       'VEHICLE TYPE CODE 1' : 'veh_type_1',
                       'VEHICLE TYPE CODE 2' : 'veh_type_2',
                       'VEHICLE TYPE CODE 3' : 'veh_type_3',
                       'VEHICLE TYPE CODE 4' : 'veh_type_4',
                       'VEHICLE TYPE CODE 5' : 'veh_type_5'},
           inplace = True)

In [ ]:
# ((Codeblock Five))

# Find all the keys with missing values and validate our data model changes
df.isna().sum()

## Analyst Notes

> Assign missing Borough records to the value of 'NYC'

Borough and Zip Code are missing for ~600k records, which is ~30% of the data. This is a significant portion, so we can't disregard it. We'll have 5 boroughs plus 'NYC' to tag the data with.

In [ ]:
# ((Codeblock Six))

# Fill all blank values in column Borough
# If a value is NaN it will be NYC
df.loc[df['borough'].isnull(), 'borough'] = 'NYC'

# View the output;`borough` should have 0 NaN values
df.isna().sum()

## Analyst Notes
 
> Filter Out Total Injured and Total Fatality NaN values

We should only keep records where the total number of injured and killed is greater than 0. This will ensure that we're focusing on relevant incidents in our analysis.

In [ ]:
# ((Codeblock Seven))

# Remove NaN from TOTAL INJURED
df = df.dropna(axis=0, subset=['total_injured'])

In [ ]:
# ((Codeblock Eight))

# Remove NaN from TOTAL KILLED
df = df.dropna(axis=0, subset=['total_fatality'])

In [ ]:
# ((Codeblock Nine))

# If we are interested in collisions that have injuries, then we may  want to keep those values > 0 as df1
df1 = df[(df['total_injured'] > 0)]

In [ ]:
# ((Codeblock Ten))

# If we are interested in the collisions that have fatalities, then we may want to keep those values > 0 as df2
df2 = df[(df['total_fatality'] > 0)]

In [ ]:
# ((Codeblock Eleven))

# To keep only those records with either injuries or fatalities we can  now concatenate df1 and df2 and put it back as df
df = pd.concat([df1, df2])

In [ ]:
# ((Codeblock Twelve))

# Combine DATE and TIME column to transform Series to DateTime needed for further analysis
df['date'] = df['date'] + ' ' + df['time']

In [ ]:
# ((Codeblock Thirteen))

# Convert string to DateTime
df['date'] = pd.to_datetime(df.date)

In [ ]:
# ((Codeblock Fourteen))

# Year filter
df['year'] = pd.to_datetime(df['date']).dt.year

In [ ]:
# ((Codeblock Fifteen))

# Quarter filter
df['quarter'] = pd.to_datetime(df['date']).dt.quarter

In [ ]:
# ((Codeblock Sixteen))

# Month filter
df['month'] = pd.to_datetime(df['date']).dt.month

In [ ]:
# ((Codeblock Seventeen))

# Day of the week filter
df['weekday'] = pd.to_datetime(df['date']).dt.weekday

In [ ]:
# ((Codeblock Eighteen))

#Fill in missing values with Empty
df = df.fillna(value='EMPTY')

In [ ]:
# ((Codeblock Nineteenth))

# Validate the final dataset before analysis
df.info(verbose = True, show_counts = True)
# We have ~430k relevant records instead of 1.9 million and our file is ~100 MB from ~400 MB at the beginning of preparation
# This file is in a much better state to ingest into LogScale and begin working with a sample of the data.
# Additionally, we have are steps to clean the data within LogScale via parser as ingest is streaming via API.

In [ ]:
# ((Codeblock Twenty))

# Save the cleaned dataframe to a new CSV File
df.to_csv("/Users/Administrator/Documents/clean_nyc_collisions.csv", index=False)